# 📘 Word 文档 → 文档代码化写作 Demo
本 Notebook 演示如何将 Word 文档内容结构化，并用代码自动生成文档。

In [1]:
# 安装依赖（首次运行时需要）
!pip install pyyaml jinja2

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)

   ---------------------------------------- 0/3 [pyyaml]
   -------------------------- ------------- 2/3 [jinja2]
   -------------------------- ------------- 2/3 [jinja2]
   ---------------------------------------- 3/3 [jinja2]



## 第一步：准备数据 (products.yaml)
在传统 Word 文档里，你可能直接写：
```
N706B 模块 AT 指令手册
波特率：115200
命令：AT+CSQ → 查询信号强度
```
现在我们把它转成结构化数据：

这段代码**使用了 Python 的 PyYAML 库来创建并读取一个 YAML 格式的配置文件，目的是将嵌套的、易读的文本内容转化为程序容易处理的结构化数据**（如 Python 中的字典和列表）。<"隐性知识显性化">

举个具体场景：若 N706B 模块新增 10 条 AT 命令，结构化方案只需在 YAML 中追加 10 个 commands 条目，Sphinx 会自动更新文档的目录、索引和交叉引用；而 Word 方案需要手动修改多个章节，还可能遗漏格式一致性。

预处理过程：
如果 Word 内容体量很大（比如数百条命令），完全手动转 YAML 确实低效，可考虑中间过渡方案：
Word → 表格 → YAML   “借助多维表格去维护”
先在 Word 中将命令按"产品型号/命令名称/描述/示例"整理成规范表格
导出为 CSV，再用 Python 脚本批量转换为 YAML（示例代码片段）：

import pandas as pd
df = pd.read_csv("commands.csv")
yaml_data = {"products": df.to_dict("records")}  # 自动转为字典结构

In [2]:
import yaml

yaml_content = """
products:
  - name: N706B
    version: V1.0
    baud_rate: 115200
    commands:
      - cmd: AT+CSQ
        desc: 查询信号强度
        example: "+CSQ: 25,0"
  - name: N58-GL
    version: V2.0
    baud_rate: 921600
    commands:
      - cmd: AT+CGATT
        desc: 网络附着
        example: "+CGATT: 1"
"""

with open("products.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_content)

data = yaml.safe_load(yaml_content)
data

{'products': [{'name': 'N706B',
   'version': 'V1.0',
   'baud_rate': 115200,
   'commands': [{'cmd': 'AT+CSQ', 'desc': '查询信号强度', 'example': '+CSQ: 25,0'}]},
  {'name': 'N58-GL',
   'version': 'V2.0',
   'baud_rate': 921600,
   'commands': [{'cmd': 'AT+CGATT', 'desc': '网络附着', 'example': '+CGATT: 1'}]}]}

## 第二步：准备模板
用 Jinja2 模板定义文档的结构，而不是直接在 Word 里排版。

In [3]:
from jinja2 import Template

template_str = """
{{ name }} 模块 AT 指令手册 (版本 {{ version }})
========================================

基础信息
--------
* 型号: {{ name }}
* 波特率: {{ baud_rate }}

支持命令
--------
| 指令   | 功能描述       | 示例响应 |
|--------|----------------|----------|
{% for c in commands %}| {{ c.cmd }} | {{ c.desc }} | {{ c.example }} |
{% endfor %}
"""

template = Template(template_str)

## 第三步：生成文档
循环读取 YAML 数据，用模板渲染输出 Markdown 文本。

In [4]:
for product in data["products"]:
    output = template.render(**product)
    filename = f"{product['name']}_manual.md"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)
    print(f"生成文档: {filename}\n")
    print(output)
    print("-"*40)

生成文档: N706B_manual.md


N706B 模块 AT 指令手册 (版本 V1.0)

基础信息
--------
* 型号: N706B
* 波特率: 115200

支持命令
--------
| 指令   | 功能描述       | 示例响应 |
|--------|----------------|----------|
| AT+CSQ | 查询信号强度 | +CSQ: 25,0 |

----------------------------------------
生成文档: N58-GL_manual.md


N58-GL 模块 AT 指令手册 (版本 V2.0)

基础信息
--------
* 型号: N58-GL
* 波特率: 921600

支持命令
--------
| 指令   | 功能描述       | 示例响应 |
|--------|----------------|----------|
| AT+CGATT | 网络附着 | +CGATT: 1 |

----------------------------------------


## 第四步：下一步拓展
- 现在生成的文档是 Markdown (`.md`)，可以用 Pandoc 转 PDF：
  ```bash
  pandoc N706B_manual.md -o N706B_manual.pdf
  ```
- 也可以用 Sphinx/DITA-OT 来生成 HTML 和 PDF。
- 这就是 **结构化编程 + 文档代码化** 的最小实践。